In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [5]:
def read_excel():
    df_c = pd.read_excel('Resultados_2da_vuelta_Version_PCM.xlsx')
    df = df_c.copy()[:-1]
    df.fillna(0.0, inplace = True)
    df['MESA_DE_VOTACION'] = df['MESA_DE_VOTACION'].apply(lambda x: int(x))
    df['UBIGEO'] =df['UBIGEO'].apply(lambda x: ubigeo_to_string(x))
    df['UBIGEO_prov'] = df['UBIGEO'].apply(lambda x : x[:4])
    df['UBIGEO_depa'] = df['UBIGEO'].apply(lambda x: x[:2])
    df.drop(['VOTOS_VI'], axis = 1, inplace = True)
    df['VOTOS_VAL_P1_%'] = df['VOTOS_P1'] * 100 / (df['N_CVAS'] - df['VOTOS_VN'] - df['VOTOS_VB'])
    #df['VOTOS_VAL_P2_%'] = df['VOTOS_P2'] * 100 / (df['N_CVAS'] - df['VOTOS_VN'] - df['VOTOS_VB'])
    df = df[df['DESCRIP_ESTADO_ACTA'] != 'ANULADA']
    df.dropna(subset = ['VOTOS_VAL_P1_%'], inplace=True)
    return df

def new_df(df, ubi: str, tipo_ubi = 'depa'):
    if tipo_ubi == 'depa':
        new_df = df[df['UBIGEO_depa'] == ubi]
    elif tipo_ubi == 'prov':
        new_df = df[df['UBIGEO_prov'] == ubi]
    elif tipo_ubi == 'dist':
        new_df = df[df['UBIGEO'] == ubi]
        
    votos_p1_lugar = np.sum(new_df['VOTOS_P1']) * 100 / np.sum(new_df['N_CVAS'] - new_df['VOTOS_VB'] - new_df['VOTOS_VN'])
    return new_df, votos_p1_lugar

def ubigeo_to_string(x):
    string = str(int(x))
    if len(string) == 5:
        string = '0' + string
        return string
    return string

def estandarizar_columna(df, columna, columna_nueva):
    scaler = StandardScaler()
    new_data = np.array(df[columna])
    new_data = new_data.reshape(-1, 1)
    new_data_plot = scaler.fit_transform(new_data)
    new_data = new_data_plot.reshape(1, -1)[0]
    df[columna_nueva] = new_data
    return df

def graficar_box_y_hist(df, columna):
    data = df[columna]
    fig = plt.figure(figsize = (4, 3))
    ax = fig.add_axes([0,0,1,1])
    bp = ax.boxplot(data, vert = 0)
    ax.set_xlabel('Fuerza Popular -> Perú Libre')
    plt.show()

    sns.displot(data)
    plt.show()
    
    
def normalize_df(df):
    votos_val_p1_ubigeo_dist = {}
    votos_val_p1_ubigeo_prov = {}
    votos_val_p1_ubigeo_depa ={}
    for ubigeo in df['UBIGEO'].unique():
        _, votos_val_p1_lug = new_df(df, ubigeo, tipo_ubi = 'dist')
        votos_val_p1_ubigeo_dist[ubigeo] = votos_val_p1_lug
        _, votos_val_p1_lug = new_df(df, ubigeo[:4], tipo_ubi = 'prov')
        votos_val_p1_ubigeo_prov[ubigeo[:4]] = votos_val_p1_lug
        _, votos_val_p1_lug = new_df(df, ubigeo[:2], tipo_ubi = 'depa')
        votos_val_p1_ubigeo_depa[ubigeo[:2]] = votos_val_p1_lug
        
    df['VOTOS_VAL_POR_P1_Dist'] = df['UBIGEO'].apply(lambda x: votos_val_p1_ubigeo_dist.get(x))
    df['VOTOS_VAL_POR_P1_Prov'] = df['UBIGEO_prov'].apply(lambda x: votos_val_p1_ubigeo_prov.get(x))
    df['VOTOS_VAL_POR_P1_Depa'] = df['UBIGEO_depa'].apply(lambda x: votos_val_p1_ubigeo_depa.get(x))
    df['Diferencia_Dist'] = df['VOTOS_VAL_P1_%'] - df['VOTOS_VAL_POR_P1_Dist']
    df['Diferencia_Prov'] = df['VOTOS_VAL_P1_%'] - df['VOTOS_VAL_POR_P1_Prov']
    df['Diferencia_Depa'] = df['VOTOS_VAL_P1_%'] - df['VOTOS_VAL_POR_P1_Depa']
    df_aux = estandarizar_columna(df, 'Diferencia_Dist', 'Estandarizado_Dist')
    df_aux = estandarizar_columna(df_aux, 'Diferencia_Prov', 'Estandarizado_Prov')
    df_aux = estandarizar_columna(df_aux, 'Diferencia_Depa', 'Estandarizado_Depa')
    df_aux['Outliers_Dist'] = df_aux['Estandarizado_Dist'].apply(lambda x: outliers(x))
    df_aux['Outliers_Prov'] = df_aux['Estandarizado_Prov'].apply(lambda x: outliers(x))
    df_aux['Outliers_Depa'] = df_aux['Estandarizado_Depa'].apply(lambda x: outliers(x))
    return df_aux

def outliers(num):
    if num > 2 and num < 3:
        string = 'Moderado_P1'
    elif num >= 3:
        string = 'Extremo_P1'
    elif num < -2 and num >= -3:
        string = 'Moderado_P2'
    elif num < -3:
        string = 'Extremo_P2'
    else:
        string = 'Normal'
    return string

def outlier_de_df(df, tipo_outlier, numero = True):
    extremo_pl = np.sum(df[tipo_outlier] == 'Extremo_P1') 
    moderado_pl = np.sum(df[tipo_outlier] == 'Moderado_P1')
    normales = np.sum(df[tipo_outlier] == 'Normal')
    moderado_fp = np.sum(df[tipo_outlier] == 'Moderado_P2')
    extremo_fp = np.sum(df[tipo_outlier] == 'Extremo_P2')
    lista = [extremo_fp, moderado_fp, normales, moderado_pl, extremo_pl]
    total = np.sum(lista)
    porcentaje = list(np.round(np.array(lista) / total * 100, 2))
    if numero == True:
        return lista
    else:
        return porcentaje

def outlier_de_df(df, tipo_outlier, numero = True):
    extremo_pl = np.sum(df[tipo_outlier] == 'Extremo_P1') 
    moderado_pl = np.sum(df[tipo_outlier] == 'Moderado_P1')
    normales = np.sum(df[tipo_outlier] == 'Normal')
    moderado_fp = np.sum(df[tipo_outlier] == 'Moderado_P2')
    extremo_fp = np.sum(df[tipo_outlier] == 'Extremo_P2')
    lista = [extremo_fp, moderado_fp, normales, moderado_pl, extremo_pl]
    total = np.sum(lista)
    porcentaje = list(np.round(np.array(lista) / total * 100, 2))
    if numero == True:
        return lista
    else:
        return porcentaje
        
def generar_excel_resultados_distribucion_normal(df , numero = False):
    a = np.array([0]*5).reshape(1, -1)
    ubi = []
    for ubicacion in df['UBIGEO_depa'].unique():
        ubi.append(ubicacion)
        aux = df[df['UBIGEO_depa'] == ubicacion]
        b = outlier_de_df(aux, 'Outliers_Dist', numero = numero)
        b = np.array(b).reshape(1, -1)
        a = np.concatenate([a, b], axis = 0)
    a = a[1:]

    resultados = pd.DataFrame(data = a, index = ubi, columns = ['Extremo FP', 'Moderado FP', 'Normal', 'Moderado PL', 'Extremo PL'])

    ubigeo_depa = {}
    numeros = df['UBIGEO_depa'].unique()
    lugares = df['DEPARTAMENTO'].unique()
    for i in range(len(numeros)):
        ubigeo_depa[numeros[i]] = lugares[i]

    resultados['Ubigeo'] = resultados.index
    resultados['Region'] = resultados['Ubigeo'].apply(lambda x: ubigeo_depa.get(x))

    resultados.to_excel('Resultados-Outliers-Distribucion_Normal_2.xlsx') 

In [4]:
%%time
df = read_excel()
df = normalize_df(df)

Wall time: 45.2 s


In [6]:
generar_excel_resultados_distribucion_normal(df)